In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time
from selenium import webdriver

In [2]:
# visiting the webpage
browser = Browser('chrome', executable_path='../Resources/chromedriver.exe')
url = 'https://shop.tcgplayer.com/sellers'
browser.visit(url)

In [3]:
seller = 'GAMELAND'                     # select TCGplayer seller name
pages = 3                               # set number of pages of inventory to scan
browser.fill('SellerName', seller)      # enter seller name in search box

In [4]:
# click search button and then click first search result
browser.find_by_xpath('//*[@id="maincontentinnerpadding"]/div/div/form/div/ul/li[4]/div/input').click()
browser.find_by_xpath('//*[@id="maincontentinnerpadding"]/div/div[2]/div[3]/div/div[3]/button').click()
time.sleep(3)

In [6]:
# loop through pages and scrape data for; listed card price, market price, card name, card set, and card product(mtg/pokemon)
price_list = []
for i in range(1):
    req = browser.html
    soup = BeautifulSoup(req, "html.parser")               # scrape data
    cat_tags = soup.find_all('h3', class_ ="search-result__category-name")
    set_tags = soup.find_all('h4', class_ ="search-result__subtitle")
    title_tags = soup.find_all('span', class_ = "search-result__title")
    price1_tags = soup.find_all('span', class_ = "inventory__price-with-shipping")
    price2_tags = soup.find_all('span', class_ = "search-result__market-price--value")
    for  cat, sset, titles, price1, price2 in zip( cat_tags ,set_tags, title_tags, price1_tags, price2_tags): # zip scraped lists and write df row
        row = []
        row.append(cat.text)
        row.append(sset.text)
        row.append(titles.text)
        row.append(price1.text)
        row.append(price2.text)
        price_list.append(row)             
    browser.find_by_xpath('/html/body/div[2]/div/div/section[2]/section/section/section/div[2]/div[1]/a[2]').click() # click next page
    time.sleep(4)
browser.quit   
    
                 
               

<bound method BaseWebDriver.quit of <splinter.driver.webdriver.chrome.WebDriver object at 0x0000027F719D9930>>

In [18]:
df = pd.DataFrame(price_list, columns =   ["category", "set", 'Name', 'Listed Price','Market Price'] )  #Make df
df["Market Price"] = df["Market Price"].str.replace("$", "")  # remove dollar sign
df["Listed Price"] = df["Listed Price"].str.replace("$", "")
df['Market Price'] = df['Market Price'].astype(float)         # set as float for math
df['Listed Price'] = df['Listed Price'].astype(float)
df["over_under"] = (df["Listed Price"]) - (df['Market Price'])     # create a column showing the over/under of price compared to market price
df = df.sort_values(["over_under"])    # sort values to show most under market price cards
df.head(3)

,category,set,Name,Listed Price,Market Price,over_under
18,Magic: The Gathering,Strixhaven: School of Mages,Dragon's Approach,1.00,2.92,-1.92
2,Magic: The Gathering,Theros Beyond Death,Island,1.00,1.50,-0.50
14,Magic: The Gathering,Modern Horizons,Snow-Covered Plains,0.95,1.32,-0.37


In [19]:
limit_df= df.loc[sorted_df['category'] == "Magic: The Gathering"]  # make df of only magic the gathering cards
limit_df= limit_df.iloc[:25]  # limit to the 25 most under priced cards
limit_df

,category,set,Name,Listed Price,Market Price,over_under
18,Magic: The Gathering,Strixhaven: School of Mages,Dragon's Approach,1.00,2.92,-1.92
2,Magic: The Gathering,Theros Beyond Death,Island,1.00,1.50,-0.50
14,Magic: The Gathering,Modern Horizons,Snow-Covered Plains,0.95,1.32,-0.37
19,Magic: The Gathering,Dominaria United,Forest (Showcase),0.69,0.90,-0.21
21,Magic: The Gathering,Modern Horizons,Snow-Covered Forest,1.00,1.12,-0.12
10,Magic: The Gathering,Core Set 2020,Shifting Ceratops,0.25,0.33,-0.08
15,Magic: The Gathering,Wilds of Eldraine,Mountain (0265),0.30,0.33,-0.03
3,Magic: The Gathering,Theros Beyond Death,Mountain,1.35,1.31,0.04
5,Magic: The Gathering,Unfinity,Forest (239) (Borderless),0.80,0.76,0.04
11,Magic: The Gathering,The Lost Caverns of Ixalan,Hidden Nursery,0.15,0.10,0.05


In [13]:
df.to_csv(f"{seller}_report.csv")    # save entire scaned inventory to csv to browse if desired